# SVM classifier

In [59]:
import numpy as np
import scipy.optimize
import scipy.special
import sklearn.datasets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
def load_iris_binary():
    D, L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']
    D = D[:, L != 0] # remove setosa from D
    L = L[L!=0] # remove setosa from L
    L[L==2] = 0 # We assign label 0 to virginica (was label 2)
    return D, L
def load_iris():
    D, L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']
    return D, L
def split_db_2to1(D, L, seed=0):
    nTrain = int(D.shape[1]*2.0/3.0) # 2/3 of the dataset D are used for training, 1/3 for validation
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1]) # take a random array of 150 elements, each element is 0<x<=149 (np.arange(150))
    idxTrain = idx[0:nTrain] # first 100 are indices of training samples 
    idxTest = idx[nTrain:] # remaining 50 are indices of validation samples
    DTR = D[:, idxTrain] # D for training
    DTE = D[:, idxTest] # D for validation
    LTR = L[idxTrain] # L for training
    LTE = L[idxTest] # L for validation
    return (DTR, LTR), (DTE, LTE)
D, L = load_iris_binary()
(DTR, LTR), (DTE, LTE) = split_db_2to1(D, L)

## Solution to the reformulated dual SVM (without bias term)

In [106]:
N = DTR.shape[1]
F = DTR.shape[0]
K = 1
C = 10
# Compute the labels z
LTRz = np.zeros(N)
for i in range(N):
    LTRz[i] = 1 if LTR[i]==1 else -1
LTEz = np.zeros(len(LTE))
for i in range(len(LTE)):
    LTEz[i] = 1 if LTE[i]==1 else -1
      
# Compute the expaded feature space D_ 
D_ = np.vstack((DTR, K*np.ones(N)))

# Compute matrix G_ of dot products of all samples of D_
G_ = np.dot(D_.T, D_)

# Compute matrix H_
LTRz_matrix = np.dot(LTRz.reshape(-1,1), LTRz.reshape(1,-1))
H_ = G_ * LTRz_matrix

In [65]:
# Define a function that represents J_D(alpha) we want to minimize
def LDc_obj(alpha): # alpha has shape (n,)
    n = len(alpha)
    minusJDc = 0.5 * np.dot(np.dot(alpha.T, H_), alpha) - np.dot(alpha.T, np.ones(n)) # 1x1
    return minusJDc, gradLDc(alpha)

def gradLDc(alpha):
    n = len(alpha)
    return (np.dot(H_, alpha) - 1).reshape(n)

In [66]:
# Minimize LD_(alpha)
bounds = [(0,C)] * N
m, primal, _ = scipy.optimize.fmin_l_bfgs_b(func=LDc_obj, 
                                       bounds=bounds,
                                       x0=np.zeros(N), factr=1.0)
# m is the final alpha
wc_star = np.sum(m * LTRz * D_, axis=1)

# extract w and b
w_star, b_star = wc_star[:-1], wc_star[-1]

# compute the scores
S = np.dot(w_star.T, DTE) + b_star*K # the *K is not present in slides!??
# or: S=np.dot(wc_star.T, np.vstack((DTE, K*np.ones(DTE.shape[1]))))

In [67]:
def primal_obj(wc_star):
    return 0.5 * np.linalg.norm(wc_star)**2 + C * np.sum(np.maximum(0,1-LTRz * np.dot(wc_star.T, D_)))
def duality_gap(wc_star, alpha_star):
    return primal_obj(wc_star) + LDc_obj(alpha_star)[0]
primal_loss = primal_obj(wc_star)
dual_loss = LDc_obj(m)[0]
duality_gap=primal_obj(wc_star) + LDc_obj(m)[0]

In [68]:
predict_labels = np.where(S > 0, 1, 0)

In [69]:
acc = sum(predict_labels == LTE) / len(predict_labels)
print('C=%.1f, K=%d, Primal loss: %f, Dual loss: %f, Duality gap: %.9f, Error rate: %.1f%%'%(C,K,primal_loss,dual_loss,duality_gap,(1-acc)*100))

C=10.0, K=1, Primal loss: 78.968950, Dual loss: -78.968928, Duality gap: 0.000021876, Error rate: 5.9%


## Kernel SVM

In [153]:
K = eps = 1
c=1
d=2
C = 1
gamma=1
#ker = kernel(DTR, DTR, 'Polynomial', eps, c, d)
ker = kernel(DTR, DTR, 'RBF', eps, gamma)
H_ = LTRz_matrix * ker
ker

array([[2.        , 1.10539922, 1.88692044, ..., 1.06925223, 1.28938422,
        1.03508435],
       [1.10539922, 2.        , 1.17377394, ..., 1.12245643, 1.19204991,
        1.09255058],
       [1.88692044, 1.17377394, 2.        , ..., 1.10539922, 1.39851904,
        1.06521929],
       ...,
       [1.06925223, 1.12245643, 1.10539922, ..., 2.        , 1.73344696,
        1.63128365],
       [1.28938422, 1.19204991, 1.39851904, ..., 1.73344696, 2.        ,
        1.43604929],
       [1.03508435, 1.09255058, 1.06521929, ..., 1.63128365, 1.43604929,
        2.        ]])

In [145]:
def kernel(X1, X2, kernelType, *params):
    eps = params[0]
    kernel = 0;
    if kernelType == 'Polynomial':
        c = params[1]
        d = params[2]
        kernel = (np.dot(X1.T, X2) + c)**d
    elif kernelType == 'RBF':
        gamma = params[1]       
        x = np.repeat(X1, X2.shape[1], axis=1)
        y = np.tile(X2, X1.shape[1])
        kernel = np.exp(-gamma * np.linalg.norm(x-y, axis=0).reshape(X1.shape[1],X2.shape[1])**2)
    return kernel + eps

In [146]:
def LDc_obj(alpha): # alpha has shape (n,)
    n = len(alpha)
    minusJDc = 0.5 * np.dot(np.dot(alpha.T, H_), alpha) - np.dot(alpha.T, np.ones(n)) # 1x1
    return minusJDc, gradLDc(alpha)

def gradLDc(alpha):
    n = len(alpha)
    return (np.dot(H_, alpha) - 1).reshape(n)

In [147]:
bounds = [(0,C)] * N
m, primal, _ = scipy.optimize.fmin_l_bfgs_b(func=LDc_obj, 
                                            bounds=bounds,
                                            x0=np.zeros(N), 
                                            factr=1.0)
wc_star = np.sum(m * LTRz * D_, axis=1)
w_star, b_star = wc_star[:-1], wc_star[-1]
#S = np.sum(np.dot((m*LTRz).reshape(1,-1), kernel(DTR,DTE,'Polynomial', eps, c, d)), axis=0)
S = np.sum(np.dot((m*LTRz).reshape(1,-1), kernel(DTR,DTE,'RBF', eps, gamma)), axis=0)
dual_loss = LDc_obj(m)[0]
predict_labels = np.where(S > 0, 1, 0)
acc = sum(predict_labels == LTE) / len(predict_labels)
print('C=%.1f, K=sqrt(eps)=%d, Dual loss: %f, Error rate: %.1f%%'%(C,K,dual_loss,(1-acc)*100))

C=1.0, K=sqrt(eps)=1, Dual loss: -11.978133, Error rate: 8.8%


In [151]:
H_

array([[ 2.        , -1.10539922,  1.88692044, ..., -1.06925223,
        -1.28938422, -1.03508435],
       [-1.10539922,  2.        , -1.17377394, ...,  1.12245643,
         1.19204991,  1.09255058],
       [ 1.88692044, -1.17377394,  2.        , ..., -1.10539922,
        -1.39851904, -1.06521929],
       ...,
       [-1.06925223,  1.12245643, -1.10539922, ...,  2.        ,
         1.73344696,  1.63128365],
       [-1.28938422,  1.19204991, -1.39851904, ...,  1.73344696,
         2.        ,  1.43604929],
       [-1.03508435,  1.09255058, -1.06521929, ...,  1.63128365,
         1.43604929,  2.        ]])

In [110]:
m.shape

(66,)